In [1]:
import requests_cache
import requests
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
from pandas.tools.plotting import scatter_matrix
import numpy as np
import seaborn as sns
from requests_oauthlib import OAuth1Session
import glob
import json
import os
import urllib2
from urllib2 import Request, urlopen
from requests_oauthlib import OAuth1Session
from bs4 import BeautifulSoup

### Data Wrangling

In [2]:
requests_cache.install_cache("cache")

# criteria: 1=Distance, 2=Highest Rated

def yelp_search(term, location, criteria):
    """
    makes a search request
    
    Argument: search term, location
    
    Return: search result items as a list (for 0 items, return an empty list)
    """
    consumer_key = "uFKYftN0fu4pD6guW2cYng"
    consumer_secret = "iUYQ2Hk3-vTQVd0zOFnvloWPivA"
    token = "FZnep4FiFmwrrifuNm6M-_K661Wb3J02"
    token_secret = "CRvBo_fO-r4pE_-p2UrnN_mxBlc"
   
    urlbase = "http://api.yelp.com/v2/search"
    params = ({"term": term})
    
    criteria = criteria.lower()
    
    if criteria == "distance":
        params.update({"location": location, "sort": "1", "cll" : "latitude,longitude"})
    else:
        params.update({"location": location, "sort": "2", "cll" : "latitude,longitude"})
    session = OAuth1Session(consumer_key, consumer_secret,token,token_secret)
     
    request = session.get(urlbase,params=params)
   
    #Transforms JSON API response into dictionary
    data = request.json()
    session.close()
    output = data
    
    if "businesses" not in output:
        return {}
    
    else:
        return output["businesses"]

In [4]:
def extract(url):
    """
    extract
    """
    info = requests.get(url).content
    wikisoup = BeautifulSoup(info, 'lxml')
    city_data = wikisoup.findAll("th", {"scope": "row"})
    population_data = wikisoup.findAll("td", {"align":"right"})
    city = [city.text for city in city_data]
    city = city[1:-8]
    population_data = wikisoup.findAll("td", {"align":"right"})
    population_data = "\n".join([population.text for population in population_data])
    population_split = population_data.split("\n")
    population = population_split[0::2]
    info_dict = {"city": city, "population": population}
    return info_dict

In [5]:
url = "https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_California"
url_output = extract(url)
pd_scrape = pd.DataFrame(url_output)

In [6]:
# this excel provide latitude and longitude information
pd_location = pd.read_excel("/Users/Chloechen/Downloads/calcities.xlsx", header = 0)
pd_location_1 = pd_location.rename(columns = {'Name':'city'})
pd_geo = pd.merge(pd_scrape, pd_location_1, on = "city")
pd_geo['city'] = pd_geo['city'] + ', CA'

In [7]:
def search(keyterm, city, criteria):
    result = yelp_search(keyterm, city, criteria)
    df_result = pd.DataFrame(result)
    df_result["city"] = city
    return df_result

In [8]:
def create_df(keyterm, criteria):
    yelp_result = [search(keyterm, "%s, CA" %city, criteria) for city in pd_scrape["city"]]
    df_result = pd.concat(yelp_result, ignore_index = True)
    df_result_drop = df_result.drop(["deals","image_url", "location", "menu_date_updated", "menu_provider", "phone", "rating_img_url", "rating_img_url_large", "rating_img_url_small", "snippet_image_url"], 1)
    df_result_final = pd.merge(df_result_drop, pd_geo, on = "city", how = "inner")
    return df_result_final

In [9]:
def change_na(dataframe):
    #change category thats 'Nan' to 'Unknown'
    for i in range(0,len(dataframe)):
        if type(dataframe['categories'][i]) == float:
            dataframe['categories'][i] = [[u'Unknown']]
            return dataframe

In [10]:
def create_final_df(keyterm, criteria):
    dataframe = create_df(keyterm, criteria)
    final = change_na(dataframe)
    return final

In [11]:
keyterm = "restaurants"
criteria = "Distance"
create_final_df(keyterm, criteria)

,categories,display_phone,gift_certificates,id,image_url,is_claimed,is_closed,location,menu_date_updated,menu_provider,...,phone,rating,rating_img_url,rating_img_url_large,rating_img_url_small,review_count,snippet_image_url,snippet_text,url,city
0,"[[American (New), newamerican], [Desserts, des...",+1-415-621-3870,NaN,frances-san-francisco,https://s3-media3.fl.yelpcdn.com/bphoto/OwyUcI...,True,False,"{u'cross_streets': u'Pond St & Noe St', u'city...",1.472638e+09,single_platform,...,4156213870,4.5,https://s3-media2.fl.yelpcdn.com/assets/2/www/...,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,https://s3-media2.fl.yelpcdn.com/assets/2/www/...,1292,https://s3-media3.fl.yelpcdn.com/photo/sEKbJHy...,I've had numerous reservations at Frances over...,https://www.yelp.com/biz/frances-san-francisco...,San Francisco
1,"[[Mediterranean, mediterranean], [American (Tr...",+1-415-829-8383,NaN,novy-san-francisco-6,https://s3-media2.fl.yelpcdn.com/bphoto/JcA_zV...,True,False,"{u'cross_streets': u'Noe St & Castro St', u'ci...",1.481489e+09,locu,...,4158298383,4.0,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,https://s3-media2.fl.yelpcdn.com/assets/2/www/...,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,233,https://s3-media3.fl.yelpcdn.com/photo/lDy2WmU...,We can here for brunch on a Saturday and it to...,https://www.yelp.com/biz/novy-san-francisco-6?...,San Francisco
2,"[[Tapas/Small Plates, tapasmallplates]]",+1-415-643-5656,NaN,loló-san-francisco-4,https://s3-media3.fl.yelpcdn.com/bphoto/-_hqew...,True,False,"{u'cross_streets': u'21st St & Liberty St', u'...",1.472612e+09,single_platform,...,4156435656,4.0,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,https://s3-media2.fl.yelpcdn.com/assets/2/www/...,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,1556,https://s3-media2.fl.yelpcdn.com/photo/s1OGkZ7...,Colorful decor and friendly vibes here!\n\nGot...,https://www.yelp.com/biz/lol%C3%B3-san-francis...,San Francisco
3,"[[Tapas Bars, tapas], [Latin American, latin]]",+1-415-386-7670,NaN,cha-cha-cha-san-francisco-4,https://s3-media4.fl.yelpcdn.com/bphoto/POnGXZ...,True,False,"{u'cross_streets': u'Shrader St & Stanyan St',...",1.478257e+09,single_platform,...,4153867670,4.0,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,https://s3-media2.fl.yelpcdn.com/assets/2/www/...,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,2781,https://s3-media1.fl.yelpcdn.com/photo/ORAwhkw...,Came in late on a Friday night with some frien...,https://www.yelp.com/biz/cha-cha-cha-san-franc...,San Francisco
4,"[[Mediterranean, mediterranean], [Greek, greek]]",+1-415-400-5458,NaN,souvla-san-francisco-3,https://s3-media3.fl.yelpcdn.com/bphoto/TNXubv...,True,False,"{u'cross_streets': u'Fell St & Hayes St', u'ci...",NaN,NaN,...,4154005458,4.5,https://s3-media2.fl.yelpcdn.com/assets/2/www/...,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,https://s3-media2.fl.yelpcdn.com/assets/2/www/...,113,https://s3-media2.fl.yelpcdn.com/photo/SA_4eO9...,"Such a cool place, good vibes. Delicious food...",https://www.yelp.com/biz/souvla-san-francisco-...,San Francisco
5,"[[Cocktail Bars, cocktailbars], [American (New...",+1-415-817-1939,NaN,horsefeather-san-francisco-2,https://s3-media3.fl.yelpcdn.com/bphoto/K1yHie...,True,False,"{u'city': u'San Francisco', u'display_address'...",NaN,NaN,...,4158171939,4.0,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,https://s3-media2.fl.yelpcdn.com/assets/2/www/...,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,80,https://s3-media4.fl.yelpcdn.com/photo/llWQWKW...,Solid place for reasonably priced fancy cockta...,https://www.yelp.com/biz/horsefeather-san-fran...,San Francisco
6,"[[American (New), newamerican], [Modern Europe...",+1-415-864-8643,NaN,nopa-san-francisco,https://s3-media1.fl.yelpcdn.com/bphoto/M8lkaD...,True,False,"{u'cross_streets': u'Fell St & Hayes St', u'ci...",1.472596e+09,single_platform,...,4158648643,4.0,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,https://s3-media2.fl.yelpcdn.com/assets/2/www/...,ht